In [1]:
#import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from googleapiclient.discovery import build
import isodate
import creds
from sqlalchemy import create_engine
import db_creds

In [2]:
api_key  = creds.api_key
#channel_id = 'UCBJycsmduvYEL83R_U4JriQ'

# get multiple channels details

channel_ids = [     
               'UCBJycsmduvYEL83R_U4JriQ', # Mkbhd
               'UCMiJRAwDNSNzuYeN2uWa0pA', # Mrwhosetheboss
               'UCSOpcUkE-is7u7c4AkLgqTw', # Mrmobile
               'UCsTcErHg8oDvUnTzoqsYeNw', # Unbox Therapy
               'UCO2WJZKQoDW4Te6NHx4KfTg', # Geekyranjit
               'UCEPL07qzVsOcHd3sMUws65g', # Trakin Tech
               'UCXUJJNoP1QupwsYIWFXmsZg', # Tech Burner
               'UCS261lQDXjMvc8Jck-2CQEg', # Gyan therapy
               'UCvpfclapgcuJo0M_x65pfRw', # Beebom
               'UCdoDYkupTElV62K7ttA_E4w', # Tech Bar
               'UClVIlK8QHZ2PFkXF97bA0lg', # C4etech
               'UCn7DATpF0Nk-YJmjw4gMwfg', # Trakin Tech English
               'UCXuqSBlHAE6Xw-yeJA0Tunw' # Linus Tech Tips
               ]

youtube = build('youtube','v3',developerKey=api_key)

### Function to get Channel's stats

In [3]:
def get_channel_stats(youtube,channel_ids):
   
    all_data = []

    request = youtube.channels().list(part='snippet,contentDetails,statistics',id=','.join(channel_ids))

    response = request.execute()
    
    for i in range(len(response['items'])):

        data = dict(channel_name = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    total_videos = response['items'][i]['statistics']['videoCount'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                )
        all_data.append(data)
    
    return all_data

In [4]:
op_data = get_channel_stats(youtube,channel_ids)

In [5]:
df = pd.DataFrame(op_data)
df

,channel_name,subscribers,views,total_videos,playlist_id
0,Geekyranjit,3310000,824479875,3294,UUO2WJZKQoDW4Te6NHx4KfTg
1,Beebom,3010000,761774065,1130,UUvpfclapgcuJo0M_x65pfRw
2,Linus Tech Tips,15400000,7337013558,6606,UUXuqSBlHAE6Xw-yeJA0Tunw
3,Unbox Therapy,20900000,4563283454,2218,UUsTcErHg8oDvUnTzoqsYeNw
4,Mrwhosetheboss,16700000,4408605230,1619,UUMiJRAwDNSNzuYeN2uWa0pA
5,TechBar,4320000,973470033,1213,UUdoDYkupTElV62K7ttA_E4w
6,Marques Brownlee,17700000,3803033579,1596,UUBJycsmduvYEL83R_U4JriQ
7,Tech Burner,11000000,1843503655,1183,UUXUJJNoP1QupwsYIWFXmsZg
8,Trakin Tech,13700000,2404393230,3729,UUEPL07qzVsOcHd3sMUws65g
9,Gyan Therapy,2830000,605002766,1118,UUS261lQDXjMvc8Jck-2CQEg


### Function to get video IDs

In [6]:
#playlist_id = df.loc[df['channel_name']=='Beebom','playlist_id'].iloc[0]

playlist_ids = []

for i in df['playlist_id'].values:
    playlist_ids.append(i)

playlist_ids   

['UUO2WJZKQoDW4Te6NHx4KfTg',
 'UUvpfclapgcuJo0M_x65pfRw',
 'UUXuqSBlHAE6Xw-yeJA0Tunw',
 'UUsTcErHg8oDvUnTzoqsYeNw',
 'UUMiJRAwDNSNzuYeN2uWa0pA',
 'UUdoDYkupTElV62K7ttA_E4w',
 'UUBJycsmduvYEL83R_U4JriQ',
 'UUXUJJNoP1QupwsYIWFXmsZg',
 'UUEPL07qzVsOcHd3sMUws65g',
 'UUS261lQDXjMvc8Jck-2CQEg',
 'UUlVIlK8QHZ2PFkXF97bA0lg',
 'UUn7DATpF0Nk-YJmjw4gMwfg',
 'UUSOpcUkE-is7u7c4AkLgqTw']

In [7]:
def get_video_ids(youtube,playlist_ids):

    video_ids = []

    for playlist_id in playlist_ids:

        request = youtube.playlistItems().list(part='contentDetails',playlistId=playlist_id,maxResults=50)

        response = request.execute()

        for i in range(len(response['items'])):
            video_ids.append(response['items'][i]['contentDetails']['videoId'])

        

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(part='contentDetails',
                                                        playlistId=playlist_id,
                                                        maxResults=50,
                                                        pageToken=next_page_token)
                response = request.execute()

                for i in range(len(response['items'])):
                    video_ids.append(response['items'][i]['contentDetails']['videoId'])
                
            next_page_token = response.get('nextPageToken')
    return video_ids

In [8]:
video_ids= get_video_ids(youtube,playlist_ids)
len(video_ids)

27784

### Function to get video details 

In [9]:
def get_video_details(youtube,video_ids):
    
    all_video_stats = [] 

    for i in range(0, len(video_ids), 50):
        requests = youtube.videos().list(part='snippet,statistics,contentDetails,topicDetails',id=','.join(video_ids[i:i+50]))
        response = requests.execute()
        
        for video in response['items']:
            video_stats = dict(channel_name = video['snippet'].get('channelTitle'),
                               title  = video['snippet'].get('title'),
                               published_date = video['snippet'].get('publishedAt'),
                               tags = video['snippet'].get('tags'),
                               views = video['statistics'].get('viewCount',0),
                               likes = video['statistics'].get('likeCount',0),
                               comments = video['statistics'].get('commentCount',0),  
                               favorite_count = video['statistics'].get('favoriteCount',0),    
                               duration = video['contentDetails'].get('duration'),
                               definition = video['contentDetails'].get('definition'),
                               caption = video['contentDetails'].get('caption')
                              )
            all_video_stats.append(video_stats)

    return all_video_stats

In [10]:
vid = get_video_details(youtube,video_ids)

In [11]:
df_vid = pd.DataFrame(vid)
df_vid.head(3)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption
0,Geekyranjit,9th Tech Q&A Session for 2023,2023-10-11T07:42:23Z,"[tech Q&A session, tech Q&A, tech AMA, tech Q&...",63818,3096,293,0,PT23M16S,hd,false
1,Geekyranjit,Moto Edge 40 Neo - The Game Changer | Premium...,2023-09-22T13:34:04Z,"[moto edge 40 neo, motorola edge 40 neo, edge ...",414891,11512,1194,0,PT15M11S,hd,false
2,Geekyranjit,LG QNED (2023) - Good 120Hz 4K Smart TV,2023-09-20T07:27:03Z,"[LG QNED, LG OLED, LG 2023, LG smart tv, LG be...",70577,1312,112,0,PT9M42S,hd,false


### Fix data type

In [12]:
df_vid.dtypes

channel_name      object
title             object
published_date    object
tags              object
views             object
likes             object
comments          object
favorite_count    object
duration          object
definition        object
caption           object
dtype: object

### Change data type of columns

In [13]:
df_vid['published_date'] = pd.to_datetime(df_vid['published_date'])
df_vid['views'] = df_vid['views'].astype(int)
df_vid['likes'] = df_vid['likes'].astype(int)
df_vid['comments'] = df_vid['comments'].astype(int)
df_vid['favorite_count'] = df_vid['favorite_count'].astype(int)

#### Convert Duration column from YT time format to Seconds

In [14]:
new_dur=[]
for i in df_vid['duration'].values:
    dur = isodate.parse_duration(i)
    new_dur.append(dur.total_seconds())

df_vid['time_seconds'] = new_dur

### Get year and month from published_dat

In [15]:
df_vid['day'] = df_vid['published_date'].dt.date
df_vid['year'] = df_vid['published_date'].dt.year
df_vid['month'] = df_vid['published_date'].dt.month
df_vid.head(3)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption,time_seconds,day,year,month
0,Geekyranjit,9th Tech Q&A Session for 2023,2023-10-11 07:42:23+00:00,"[tech Q&A session, tech Q&A, tech AMA, tech Q&...",63818,3096,293,0,PT23M16S,hd,false,1396.0,2023-10-11,2023,10
1,Geekyranjit,Moto Edge 40 Neo - The Game Changer | Premium...,2023-09-22 13:34:04+00:00,"[moto edge 40 neo, motorola edge 40 neo, edge ...",414891,11512,1194,0,PT15M11S,hd,false,911.0,2023-09-22,2023,9
2,Geekyranjit,LG QNED (2023) - Good 120Hz 4K Smart TV,2023-09-20 07:27:03+00:00,"[LG QNED, LG OLED, LG 2023, LG smart tv, LG be...",70577,1312,112,0,PT9M42S,hd,false,582.0,2023-09-20,2023,9


### Get tag count

In [16]:
df_vid['tag_count'] = df_vid['tags'].apply(lambda x : 0 if x is None else len(x))

df_vid.head(3)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption,time_seconds,day,year,month,tag_count
0,Geekyranjit,9th Tech Q&A Session for 2023,2023-10-11 07:42:23+00:00,"[tech Q&A session, tech Q&A, tech AMA, tech Q&...",63818,3096,293,0,PT23M16S,hd,false,1396.0,2023-10-11,2023,10,11
1,Geekyranjit,Moto Edge 40 Neo - The Game Changer | Premium...,2023-09-22 13:34:04+00:00,"[moto edge 40 neo, motorola edge 40 neo, edge ...",414891,11512,1194,0,PT15M11S,hd,false,911.0,2023-09-22,2023,9,10
2,Geekyranjit,LG QNED (2023) - Good 120Hz 4K Smart TV,2023-09-20 07:27:03+00:00,"[LG QNED, LG OLED, LG 2023, LG smart tv, LG be...",70577,1312,112,0,PT9M42S,hd,false,582.0,2023-09-20,2023,9,14


In [17]:
df_vid.isnull().sum()

channel_name        0
title               0
published_date      0
tags              603
views               0
likes               0
comments            0
favorite_count      0
duration            0
definition          0
caption             0
time_seconds        0
day                 0
year                0
month               0
tag_count           0
dtype: int64

In [18]:
x=df_vid[(df_vid['channel_name']=='Linus Tech Tips') & (df_vid['year']==2023)]
x.sort_values(by='day',ascending=False)

,channel_name,title,published_date,tags,views,likes,comments,favorite_count,duration,definition,caption,time_seconds,day,year,month,tag_count
4425,Linus Tech Tips,My most painful PC build is done.,2023-10-10 21:10:00+00:00,"[LAN Center, Gaming, 1U]",1280898,49141,2268,0,PT26M8S,hd,true,1568.0,2023-10-10,2023,10,3
4426,Linus Tech Tips,My Gaming Minivan is COMPLETE!,2023-10-08 16:58:48+00:00,"[Gaming, Gamer, Build, Cars, Minivan, MSI, Lap...",1234476,54816,3263,0,PT26M,hd,true,1560.0,2023-10-08,2023,10,17
4427,Linus Tech Tips,"Windows Subscriptions - WAN Show October 6, 2023",2023-10-07 06:27:25+00:00,"[pcmr, building, competition, gamers, how to, ...",668008,11619,1457,0,PT3H55M28S,hd,false,14128.0,2023-10-07,2023,10,14
4428,Linus Tech Tips,"Google's ""Trust Me Bro"" Guarantee is Tempting...",2023-10-05 08:30:23+00:00,"[pixelate, pixel 8, pixel 8 pro, google, made ...",1300092,52360,3505,0,PT15M52S,hd,true,952.0,2023-10-05,2023,10,13
4429,Linus Tech Tips,I Need a New Phone...,2023-10-03 19:09:17+00:00,"[pool water, salt water, phone, cell phone, mo...",1516819,73142,11232,0,PT14M15S,hd,true,855.0,2023-10-03,2023,10,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702,Linus Tech Tips,Nvidia is lying to you - RTX 4070 Ti Full Review,2023-01-04 14:00:00+00:00,"[nvidia, gpu, graphics card, video card, gefor...",3443719,99871,8784,0,PT22M51S,hd,true,1371.0,2023-01-04,2023,1,40
4703,Linus Tech Tips,These Gaming Mice are WEIRD…,2023-01-04 06:00:13+00:00,None,2859407,158811,1297,0,PT1M,hd,false,60.0,2023-01-04,2023,1,0
4704,Linus Tech Tips,We've made some AWFUL videos... - Reacting to ...,2023-01-03 21:48:28+00:00,"[reactrion, reacting, worst, videos, lett, ltt...",2262740,114398,4071,0,PT24M24S,hd,true,1464.0,2023-01-03,2023,1,14
4705,Linus Tech Tips,This GPU is WIRELESS?! - Retro GTX 460 WHDI,2023-01-02 20:16:01+00:00,"[gpu, graphics card, video card, wireless, whd...",3210149,104384,3928,0,PT22M3S,hd,true,1323.0,2023-01-02,2023,1,28


In [19]:
len(df_vid)

27784

### Connect to PostgreSQL database and insert the df_vid data into the table

In [20]:
username=db_creds.username
hostname=db_creds.hostname
port=db_creds.port
pwd=db_creds.password
db_name=db_creds.dbname


engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username,pwd,hostname,port,db_name))

connection = engine.connect()

df_vid.to_sql('youtube',con=engine,if_exists='replace',index=False)


# engine.commit()  ## commit the changes to the database

df.to_sql('yt_channel_stats',con=engine,if_exists='replace',index=False)
# engine.commit()

# connection.commit()  
connection.close()
